In [6]:
from pathlib import Path
from typing import Dict, Tuple
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm.notebook import tqdm
import csv
from datetime import datetime
directory_path = Path("../lightning_logs/theo_cov_012913")

# result_csv_path = Path(f"results_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.csv")
result_csv_path = Path(f"results.csv")

result_dict : Dict[Tuple[int, str], float] = {}

f = open(result_csv_path, "w")
writer = csv.writer(f)
writer.writerow(["seed","model","ofv"])
f.close()

seeds = sorted([entry for entry in directory_path.iterdir() if entry.is_dir()])
for seed in tqdm(seeds, desc='seed', leave=False):
    seed_name = seed.name
    models = sorted(list(seed.iterdir()))
    for seed in models:
        key = (seed_name, seed.name)
        try:
            version = sorted([entry for entry in seed.iterdir() if entry.is_dir()])[-1]
            event = sorted(list(version.glob('event*')))[-1]
            event_acc = EventAccumulator(str(event))
            event_acc.Reload()
            scalar_data = event_acc.Scalars("train_loss")[-1].value
            result_dict[key] = float(scalar_data)
            f = open(result_csv_path, "a")
            writer = csv.writer(f)
            writer.writerow([str(seed_name), seed.name, scalar_data])
            f.close()
        except :
            print(key)

seed:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
from dataclasses import dataclass
import pandas as pd

@dataclass
class OFVs :
    base_model : float
    cov_1_2_3_model : float
    cov_1_2_model :float
    cov_1_3_model : float
    cov_2_3_model : float
    cov_1_model : float
    cov_2_model : float
    cov_3_model : float

    def compare_ofv(self, less_cov_model, more_cov_model):
        assert more_cov_model <= less_cov_model or  more_cov_model - less_cov_model < 3.84, print(more_cov_model-less_cov_model)

    def check_ofv(self):
        self.compare_ofv(self.cov_1_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_2_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_3_model, self.cov_1_2_3_model)

        self.compare_ofv(self.cov_1_2_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_1_3_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_2_3_model, self.cov_1_2_3_model)

        self.compare_ofv(self.cov_1_model, self.cov_1_2_model)
        self.compare_ofv(self.cov_1_model, self.cov_1_3_model)
                         
        self.compare_ofv(self.cov_2_model, self.cov_1_2_model)
        self.compare_ofv(self.cov_2_model, self.cov_2_3_model)

        self.compare_ofv(self.cov_3_model, self.cov_1_3_model)
        self.compare_ofv(self.cov_3_model, self.cov_2_3_model)

        self.compare_ofv(self.base_model, self.cov_1_model)
        self.compare_ofv(self.base_model, self.cov_2_model)
        self.compare_ofv(self.base_model, self.cov_3_model)
    
    # def backward_elimination(self):
    #     step1 = [1,2,3]
    #     step2 = [[2,3],[1,3],[1,2]]
    #     for i in step1:
    #         for j in step2:
                

# results = Path(".").glob("result")
# result_path = sorted(list(results))[-1]
df =pd.read_csv("results.csv")
df_by_model = df.groupby('seed')
error_list = []
for seed, d in df_by_model:
    try :
        ofv_dict = {}
        for _, row in d.iterrows():
            ofv_dict[row['model']] =row['ofv']
        ofvs = OFVs(**ofv_dict)
        ofvs.check_ofv()
    except :
        print(seed)
        error_list.append(seed)

error_list

120.10925292968739
seed_1


['seed_1']